 # Segmenting and Clustering Neighborhoods in Toronto

This code will import pandas library and use .read_html() to scrape the Wikipedia page containing Toronto postal codes, boroughs and neighbourhood.

In [50]:
import pandas as pd  #import pandas library

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  #assign wikipedia page containing table to url
dfs = pd.read_html(url)
df=dfs[0]

#inspect data
print(df.shape)
df.head(4) 

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


The code below assigns rows where borough are assigned to df_ba.

In [51]:
df_ba = df[df['Borough']!='Not assigned'] 

The code below resets the index and inspects the dataframe.

In [52]:
df_ba.reset_index(inplace=True)
df_ba=df_ba.drop(columns=['index'])
df_ba.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


The code below prints the number of rows and columns in the dataframe.

In [53]:
df_ba.shape

(103, 3)

The code below loads the geospatial data in a dataframe and inspects the dataframe.

In [54]:
geo_coord = pd.read_csv('https://cocl.us/Geospatial_data')
print(geo_coord.head())
geo_coord.shape

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


(103, 3)

The code below merged the two dataframe on postal code and inspects the new dataframe.

In [55]:
df = pd.merge(df_ba,geo_coord, on='Postal Code',how='inner')
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
